In [1]:
!pip install --quiet arcgis
!pip install --quiet ipython
!pip install --quiet ipywidgets 

## Setup supporting packages

In [2]:
from IPython.display import display, HTML

from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.geocoding import geocode
import pandas as pd

import requests

## HUC12 of interest

In [3]:
#huc12 = '051302030106' # Tennessee
#huc12 = '020700100204' #DC
#huc12 = '020700100103' #DC and VA
#huc12 = '031501060703' #Alpine, al
huc12 = '071000041003' #Des Moines, Iowa

# Query ArcGIS based watershed web service

In [4]:
huc_result = None

huc_ags_sevice_url = 'https://inlandwaters.geoplatform.gov/arcgis/rest/services/NHDPlus/WatershedBoundaryDataset/MapServer/10'
huc12_conus = FeatureLayer(huc_ags_sevice_url)

huc_result = huc12_conus.query(where="HUC12='" + huc12 + "'", 
                                    out_fields='HUC12,Name,AREAACRES,AREASQKM,STATES',out_sr=4326)

if huc_result == None or huc_result.sdf.empty:
    print("\nUnable to locate HUC12 information for " + huc12)
else:    
    display(huc_result.sdf)

,AREAACRES,AREASQKM,HUC12,NAME,OBJECTID,SHAPE,STATES
0,20123.473587,81.4368,071000041003,Saylor Creek-Des Moines River,49388,"{""rings"": [[[-93.6192024086201, 41.72991864688...",IA


## Display watershed result on map

In [5]:
if huc_result == None:
    print("\nSkipping as there is no data for HUC12= " + huc12)
else:
    gis = GIS()
    map1 = gis.map()
    map1.basemap = "gray"
    map1.height = '650px'
    map1.clear_graphics()

    symbol = {
      "type": "esriSFS",
      "color": [230, 76, 0, 255],
      "outline": {
        "type": "esriSLS",
        "color": [0, 0, 0, 255],
        "width": 0.75,
        "style": "esriSLSSolid"
      },
      "style": "esriSFSSolid"
    }

    map1.draw(huc_result,symbol=symbol)
    map1.extent = huc_result.sdf.spatial.full_extent

    display(map1)
        
    huc_alpha_symbol = {
      "type": "esriSFS",
      "color": [230, 76, 0, 50],
      "outline": {
        "type": "esriSLS",
        "color": [0, 0, 0, 255],
        "width": 0.75,
        "style": "esriSLSSolid"
      },
      "style": "esriSFSSolid"
    }

MapView(layout=Layout(height='400px', width='100%'))

## Retrieve assessment units from ATTAINS HUC12 web service

In [6]:
url = 'https://attains.epa.gov/attains-public/api/huc12summary?huc=' + huc12
print("\nWeb service url = " + url)
response = requests.get(url)        # To execute get request 
#print(response.status_code)     # To print http response code  
#print(response.text)            # To print formatted JSON response 

data = response.json()

assessmentUnits = data['items'][0]['assessmentUnits']

assessmentUnitList = []
for unit in assessmentUnits:
    assessmentUnitList.append(unit['assessmentUnitId'])

assessmentUnitList = ",".join(map(lambda x: "'" + str(x) + "'",assessmentUnitList))

if assessmentUnitList == "":
    print("\nNo assessment units for HUC = " + huc12)
else:
    print("\nThe following asseessment untis where found for HUC = " + huc12 + "\n")
    print(assessmentUnitList)


Web service url = https://attains.epa.gov/attains-public/api/huc12summary?huc=071000041003

The following asseessment untis where found for HUC = 071000041003

'IA 04-UDM-6316','IA 04-UDM-1211','IA 04-UDM-1210'


## Retrieve Org ID information for Assessments

In [7]:
url = 'https://attains.epa.gov/attains-public/api/assessmentUnits?assessmentUnitIdentifier=' 
url = url + requests.utils.quote(str(assessmentUnitList).replace("'",""));

response = requests.get(url) 
data = response.json()

orgassmnt = [];
for org in data["items"]:
    for assmnt in org["assessmentUnits"]:
       orgassmnt.append((org["organizationIdentifier"],assmnt["assessmentUnitIdentifier"]))

orgassmnt

[('21IOWA', 'IA 04-UDM-1210'),
 ('21IOWA', 'IA 04-UDM-6316'),
 ('21IOWA', 'IA 04-UDM-1211')]

## Get the auid level information

In [11]:
url = 'https://attains.epa.gov/attains-public/api/assessments?'; 

columns = ['ORGID','AUID','ParameterStatusName','ParameterName'];
df = pd.DataFrame(columns=columns);
rowi = 0;

for item in orgassmnt:
    response = requests.get(url + "organizationId=" + requests.utils.quote(item[0]) + "&assessmentUnitIdentifier=" + requests.utils.quote(item[1])); 
    data = response.json();
    
    org = data["items"][0];
                            
    for assmnt in org["assessments"]:
        
        for parameter in assmnt["parameters"]:
            
            df.loc[rowi] = [
                 org["organizationIdentifier"]
                ,assmnt["assessmentUnitIdentifier"]
                ,parameter["parameterStatusName"]
                ,parameter["parameterName"]
            ];
            rowi = rowi + 1;
        
df

,ORGID,AUID,ParameterStatusName,ParamterName
0,21IOWA,IA 04-UDM-1210,Cause,ESCHERICHIA COLI (E. COLI)
1,21IOWA,IA 04-UDM-1211,Cause,NITRATE
2,21IOWA,IA 04-UDM-1211,Cause,ESCHERICHIA COLI (E. COLI)
